In [19]:
import os
import sys
import urllib.request
import datetime
import time
import json
import csv # CSV 파일 저장을 위해 라이브러리 임포트

# 네이버 API ID와 Secret Key를 입력하세요.
client_id = 'ZfccUp_ffKHgJjW7lrhf'
client_secret = 'vCFkbjZ7PT'

# [CODE 1]
# API URL에 요청을 보내고 응답을 받아오는 함수
def getRequestUrl(url):
    # API 요청 객체 생성
    req = urllib.request.Request(url)
    # 요청 헤더에 Client ID와 Secret Key 추가
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        # 요청을 보내고 응답 객체를 받음
        response = urllib.request.urlopen(req)
        # 응답 상태 코드가 200 (성공)이면
        if response.getcode() == 200:
            # 요청 성공 메시지 출력
            print("[%s] Url Request Success" % datetime.datetime.now())
            # UTF-8로 디코딩된 응답 본문을 반환
            return response.read().decode('utf-8')
    except Exception as e:
        # 예외 발생 시 에러 메시지 출력
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [10]:
# [CODE 2]
# 네이버 검색 API를 호출하고 결과를 JSON 형태로 받아오는 함수
def getNaverSearch(node, srcText, start, display):
    # API 기본 URL
    base = "https://openapi.naver.com/v1/search"
    # 검색 대상(node)에 따라 URL 경로 설정 (예: /blog.json)
    node = "/%s.json" % node
    # 검색어(srcText), 시작 위치(start), 한 번에 가져올 개수(display)를 포함한 파라미터 생성
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    # 최종 API URL 생성
    url = base + node + parameters

    # [CODE 1] 함수를 호출하여 API 요청 후 응답을 받음
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        # 응답이 정상이면 JSON 형태로 파싱하여 반환
        return json.loads(responseDecode)


In [8]:
# [CODE 3]
# 뉴스 검색 결과를 파싱하여 리스트에 추가하는 함수
def getPostData(post, jsonResult, cnt):
    # 각 뉴스 항목에서 필요한 데이터 추출
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    # 날짜 형식 변환 (API 기본 형식 -> 'YYYY-MM-DD HH:MM:SS')
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    # 추출한 데이터를 딕셔너리 형태로 만들어 jsonResult 리스트에 추가
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
                       'org_link':org_link, 'link': link, 'pDate':pDate})
    return

# 블로그 검색 결과를 파싱하여 리스트에 추가하는 함수
def getPostData_Blog(post, jsonResult, cnt):
    # 각 블로그 게시물에서 필요한 데이터 추출
    title = post['title']
    description = post['description']
    # 'bloggerlink'는 블로그의 메인 주소
    org_link = post['bloggerlink']
    # 'link'는 해당 게시물의 고유 주소
    link = post['link']

    # 날짜 형식 변환 ('YYYYMMDD' -> 'YYYY-MM-DD HH:MM:SS')
    pDate = datetime.datetime.strptime(post['postdate'], '%Y%m%d')
    pDate = pDate.strftime('%Y-MM-DD 00:00:00') # 시간 정보가 없으므로 00:00:00으로 설정

    # 추출한 데이터를 딕셔너리 형태로 만들어 jsonResult 리스트에 추가
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
                       'org_link':org_link, 'link': link, 'pDate':pDate})
    return

In [20]:
# [CODE 0]
# 스크립트의 메인 실행 함수
def main():
    # 교체1
    # node = 'blog' # 크롤링할 대상 (뉴스 검색 시 'news'로 변경)
    node = 'news' # 크롤링할 대상 (뉴스 검색 시 'news'로 변경)
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    # 처음 API를 호출하여 전체 검색 결과 수(total)를 가져옴
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    # 네이버 API는 한 번에 최대 1000건까지만 조회를 허용함
    total = jsonResponse['total'] if jsonResponse['total'] < 1000 else 1000

    # API 응답이 있고, 결과가 존재할 때까지 반복
    while ((jsonResponse != None) and (jsonResponse['display'] != 0) and cnt < total):
        for post in jsonResponse['items']:
            cnt += 1
            # 블로그 데이터를 파싱하여 jsonResult 리스트에 추가
            # getPostData_Blog(post, jsonResult, cnt) #[CODE 3]
             # 교체2
            # getPostData , 뉴스 형식
            getPostData(post, jsonResult, cnt)
        # 다음 페이지를 요청하기 위해 시작 위치(start)를 업데이트
        start = jsonResponse['start'] + jsonResponse['display']
        # 다음 페이지의 검색 결과를 요청
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

    print('전체 검색 : %d 건' % total)

    # --- CSV 파일 저장 부분 ---
    # 파일명 생성 (예: 맛집_naver_blog.csv)
    csv_filename = '%s_naver_%s.csv' % (srcText, node)

    # 쓰기 모드로 CSV 파일 열기
    with open(csv_filename, 'w', encoding='utf-8-sig', newline='') as outfile:
        # CSV 작성 객체 생성
        writer = csv.writer(outfile)
        # CSV 헤더(제목 행) 작성
        # 블로그 형식
        # writer.writerow(['번호', '제목', '요약', '블로그주소', '게시물링크', '게시일'])
        # 교체3
        writer.writerow(['번호', '제목', '요약', '원본링크', '네이버뉴스링크', '게시일'])

        # jsonResult 리스트를 순회하며 각 항목을 CSV 파일에 한 행씩 작성
        for item in jsonResult:
            writer.writerow([item['cnt'], item['title'], item['description'], item['org_link'], item['link'], item['pDate']])

    # --- JSON 파일 저장 부분 ---
    # 파일명 생성 (예: 맛집_naver_blog.json)
    json_filename = '%s_naver_%s.json' % (srcText, node)

    with open(json_filename, 'w', encoding='utf8') as outfile:
        # JSON 형식으로 변환 (읽기 좋게 들여쓰기 및 한글 인코딩 설정)
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print(f"'{json_filename}' SAVED")
    print(f"'{csv_filename}' SAVED") # CSV 저장 완료 메시지 추가

if __name__ == '__main__':
    main()

[2025-09-08 12:26:33.864255] Url Request Success
[2025-09-08 12:26:34.012903] Url Request Success
[2025-09-08 12:26:34.166547] Url Request Success
[2025-09-08 12:26:34.337605] Url Request Success
[2025-09-08 12:26:34.496719] Url Request Success
[2025-09-08 12:26:34.660051] Url Request Success
[2025-09-08 12:26:34.826447] Url Request Success
[2025-09-08 12:26:34.991487] Url Request Success
[2025-09-08 12:26:35.167255] Url Request Success
[2025-09-08 12:26:35.387179] Url Request Success
HTTP Error 400: Bad Request
[2025-09-08 12:26:35.453413] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EB%B6%80%EC%82%B0%EC%84%9C%EB%A9%B4%EB%A7%9B%EC%A7%91&start=1001&display=100
전체 검색 : 1000 건
가져온 데이터 : 1000 건
'부산서면맛집_naver_news.json' SAVED
'부산서면맛집_naver_news.csv' SAVED
